In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(SCRIPT_DIR)

from time import perf_counter
import tensorflow as tf
import numpy as np
import gpflow

from tueplots import bundles

from experiments.uci import datasets as ds 
from pathlib import Path

from rcgp.rcgp import RCSGPR
from rcgp.w import IMQ

import matplotlib.pyplot as plt
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

In [5]:
seed = 10

np.random.seed(seed)
tf.random.set_seed(seed)

n_points = 15000
sigma_n = 0.1

lengthscale = 1
variance = 1

kernel=gpflow.kernels.SquaredExponential(lengthscales=lengthscale, variance=variance)

x = np.linspace(0, 20, n_points).reshape(n_points, 1)
f = np.random.multivariate_normal(mean=np.zeros(len(x)) ,cov=kernel(x, x)).reshape(n_points, 1)

In [5]:
ratios = [0.05, 0.1, 0.2, 0.3, 0.4 ,0.5 ,0.6, 0.7, 0.8, 0.9, 1.0]
metrics = {'rmse': {'SVGP':[], 'RCSVGP':[]}, 
           'mae':{'SVGP':[], 'RCSVGP':[]}, 
           'time': {'SVGP':[], 'RCSVGP':[]},
           'time_iter': {'SVGP':[], 'RCSVGP':[]}}


lengthscale_0 = 5
variance_0 = 5
variance_out = 3
reps = 10

for percent in ratios:
    print('{} number of observations'.format(percent*n_points))
    times = {'SVGP':[], 'RCSVGP':[]}
    maes = {'SVGP':[], 'RCSVGP':[]}
    rmses = {'SVGP':[], 'RCSVGP':[]}
    times_iter = {'SVGP':[], 'RCSVGP':[]} 
    for i in range(reps):
        i_obs = np.random.choice(
            np.arange(0, n_points, 1), int(percent * n_points), replace=False)

        y_obs = f[i_obs] + np.random.normal(scale=sigma_n, size=len(i_obs)).reshape(len(i_obs), 1)
        x_obs = x[i_obs]
        n_obs = len(y_obs)

        arr1inds = x_obs[:,0].argsort()
        x_obs = x_obs[arr1inds]
        y_obs = y_obs[arr1inds]

        i_mis = np.random.choice(
        np.arange(0, n_obs, 1), int(0.05 * n_obs), replace=False)

        y_obs[i_mis]  = y_obs[i_mis]  + np.random.normal(scale=variance_out, size=len(i_mis)).reshape(-1,1)
        
        n_inducing = int(np.sqrt(n_obs))
        inducing_variable = np.linspace(0, 20, n_inducing).reshape(-1,1)

        t_before = perf_counter()
        model = gpflow.models.SGPR(
        (x_obs, y_obs),
        kernel=gpflow.kernels.SquaredExponential(lengthscales=lengthscale_0, variance=variance_0),
        noise_variance=sigma_n**2,
        inducing_variable = inducing_variable
    )
        gpflow.set_trainable(model.likelihood.variance, False)

        opt = gpflow.optimizers.Scipy()
        log = opt.minimize(model.training_loss_closure(), model.trainable_variables)
        f_mean, _ = model.predict_f(x, full_cov=False)
        t_after = perf_counter()
        n_iter = log.nit
        error = f - f_mean
        times['SVGP'].append(t_after-t_before)
        maes['SVGP'].append(np.average(np.abs(error)))
        rmses['SVGP'].append(np.average(error ** 2) ** 0.5)
        times_iter['SVGP'].append((t_after-t_before)/n_iter)

        t_before = perf_counter()
        model = RCSGPR(
        (x_obs, y_obs),
        kernel=gpflow.kernels.SquaredExponential(lengthscales=lengthscale_0, variance=variance_0),
        weighting_function=IRQ(C=np.quantile(np.abs(y_obs), 0.95)),
        noise_variance=sigma_n**2 ,
        inducing_variable = inducing_variable
    )
        gpflow.set_trainable(model.likelihood.variance, False)

        opt = gpflow.optimizers.Scipy()
        log = opt.minimize(model.training_loss_closure(), model.trainable_variables)
        f_mean, _ = model.predict_f(x, full_cov=False)
        t_after = perf_counter()
        n_iter = log.nit
        error = f - f_mean

        times['RCSVGP'].append(t_after-t_before)
        maes['RCSVGP'].append(np.average(np.abs(error)))
        rmses['RCSVGP'].append(np.average(error ** 2) ** 0.5)
        times_iter['RCSVGP'].append((t_after-t_before)/n_iter)


    metrics['time']['SVGP'].append([np.average(times['SVGP']),np.std(times['SVGP'])])
    metrics['time_iter']['SVGP'].append([np.average(times_iter['SVGP']),np.std(times_iter['SVGP'])])
    
    metrics['mae']['SVGP'].append([np.average(maes['SVGP']),np.std(maes['SVGP'])])
    metrics['rmse']['SVGP'].append([np.average(rmses['SVGP']),np.std(rmses['SVGP'])])
    print('SGP: rmse = {} \n mae = {} \n time = {}'.format(np.average(rmses['SVGP']), np.average(maes['SVGP']), np.average(times['SVGP'])))

    metrics['time']['RCSVGP'].append([np.average(times['RCSVGP']),np.std(times['RCSVGP'])])
    metrics['time_iter']['RCSVGP'].append([np.average(times_iter['RCSVGP']),np.std(times_iter['RCSVGP'])])
    
    metrics['mae']['RCSVGP'].append([np.average(maes['RCSVGP']),np.std(maes['RCSVGP'])])
    metrics['rmse']['RCSVGP'].append([np.average(rmses['RCSVGP']),np.std(rmses['RCSVGP'])])
    print('RCSVGP: rmse = {} \n mae = {} \n time = {}'.format(np.average(rmses['RCSVGP']), np.average(maes['RCSVGP']), np.average(times['RCSVGP'])))


750.0 number of observations
SGP: rmse = 0.14122307116982888 
 mae = 0.10248350387384844 
 time = 1.13647916230002
RCSVGP: rmse = 0.05421112075201575 
 mae = 0.040444687676380556 
 time = 0.9845539708992874
1500.0 number of observations
SGP: rmse = 0.11100549927215157 
 mae = 0.08252283261735352 
 time = 0.8906642876001569
RCSVGP: rmse = 0.045077109053524946 
 mae = 0.03334431745137907 
 time = 1.0650334958001622
3000.0 number of observations
SGP: rmse = 0.0997022922647009 
 mae = 0.07496752784717907 
 time = 1.3413475667002785
RCSVGP: rmse = 0.03704843246459246 
 mae = 0.028249790665219165 
 time = 1.4654654457001015
4500.0 number of observations
SGP: rmse = 0.08140622563978495 
 mae = 0.06260719322218102 
 time = 1.6216152041997702
RCSVGP: rmse = 0.03360583287726586 
 mae = 0.02486812005459251 
 time = 1.6763418792004814
6000.0 number of observations
SGP: rmse = 0.07670833312004795 
 mae = 0.059487456315108635 
 time = 2.266990324899962
RCSVGP: rmse = 0.030269880993118094 
 mae = 0.0

2023-10-13 17:35:33.704269: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:35:33.732917: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:35:33.759370: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:35:33.786521: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:35:33.814263: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular out

SGP: rmse = 0.06701516730770951 
 mae = 0.053138176828491354 
 time = 3.411628591600311
RCSVGP: rmse = 0.023920557393642725 
 mae = 0.017881805299099256 
 time = 3.065243508400454
12000.0 number of observations


2023-10-13 17:36:14.671341: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:36:14.705811: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:36:14.736605: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:36:14.769357: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:36:14.801388: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular out

SGP: rmse = 0.06782123920121771 
 mae = 0.053754085183607234 
 time = 4.30197925829998
RCSVGP: rmse = 0.026110310816895792 
 mae = 0.018305563287341446 
 time = 3.379411108400018
13500.0 number of observations


2023-10-13 17:37:47.472007: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:37:47.508713: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:37:47.546023: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:37:47.579529: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:37:47.616884: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular out

SGP: rmse = 0.06268218683332037 
 mae = 0.049243949231757504 
 time = 4.888428033299715
RCSVGP: rmse = 0.023556626551277406 
 mae = 0.016807035027327437 
 time = 4.098528370799613
15000.0 number of observations


2023-10-13 17:39:11.580471: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:39:11.627403: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:39:11.674289: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:39:11.723136: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.
2023-10-13 17:39:11.767724: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular out

SGP: rmse = 0.06279081129224026 
 mae = 0.04951564670829196 
 time = 5.528884983399985
RCSVGP: rmse = 0.024338043098961884 
 mae = 0.017408596746664177 
 time = 5.9506437834003005


In [ ]:
models = ['SVGP', 'RCSVGP']
colors = [CB_color_cycle[2],CB_color_cycle[0]]
with plt.rc_context(bundles.aistats2023()):
    fig, ax = plt.subplots(ncols=2, sharex=True,figsize=(3.25, 1.5))
    ax[0].set_ylabel('time[s]')
    ax[1].set_ylabel('RMSE')

    ax[0].set_xlabel('n observations')
    ax[1].set_xlabel('n observations')
    for i, model in enumerate(models):
        ax[1].plot(np.array(ratios)*n_points, np.array(metrics['rmse'][model])[:,0],label=model, c=colors[i], ls='--')
        ax[0].plot(np.array(ratios)*n_points, np.array(metrics['time'][model])[:,0], label=model, c=colors[i], ls='--')
    ax[1].legend()
    fig.savefig('figures/sparse.pdf', format="pdf", bbox_inches= "tight")
plt.show()

In [9]:
from scipy.stats.qmc import Halton  
def create_inducing(data):
    n = min(data.N // 2, 200)
    Z = Halton(data.D, scramble=False).random(n)
    lower = np.min(data.X, axis=0)
    upper = np.max(data.X, axis=0)
    Z = Z * (upper - lower) + lower
    return InducingPoints(Z)

In [15]:
def create_rbf(data, rng) -> Kernel:
    return gpflow.kernels.SquaredExponential(
        variance=rng.gamma(5.0, 0.2, []),
        lengthscales=rng.gamma(5.0, 0.2, [data.D]),
    )

In [31]:
no_outliers = [ds.boston.create_dataset(Path('/tmp/benchmark_data')), 
               ds.energy.create_dataset(Path('/tmp/benchmark_data')),
                ds.synthetic.create_dataset(Path('/tmp/benchmark_data')),
                  ds.yacht.create_dataset(Path('/tmp/benchmark_data'))]
time_no_outliers ={}
mae_no_outliers = {}

In [32]:
for dataset in no_outliers:
    print(dataset.name)
    time_no_outliers[dataset.name] = {'SVGP':[],'RCSVGP':[]}
    mae_no_outliers[dataset.name] = {'SVGP':[],'RCSVGP':[]}
    for i in range(10):
        print('Iteration',i)
        X_train = dataset.train.X
        Y_train = dataset.train.Y
        X_test = dataset.test.X
        Y_test = dataset.test.Y

        rng = np.random.default_rng(1235+i)

        inducing_variable = create_inducing(dataset.train)
        variance=rng.gamma(5.0, 0.2, []),
        lengthscales=rng.gamma(5.0, 0.2, [dataset.train.D]),
        noise_variance = 0.01

        t_before = perf_counter()
        model = gpflow.models.SGPR(
        (X_train, Y_train),
        kernel=create_rbf(dataset.train, rng),
        noise_variance=noise_variance,
        inducing_variable = inducing_variable
    )
        opt = gpflow.optimizers.Scipy()
        log = opt.minimize(model.training_loss_closure(), model.trainable_variables,options={"maxiter": 1_000})
        t_after = perf_counter()
        time_no_outliers[dataset.name]['SVGP'].append(t_after-t_before)

        mean, _  = model.predict_f(X_test)
        mae_no_outliers[dataset.name]['SVGP'].append(np.average(np.abs(mean-Y_test)))

        t_before = perf_counter()
        model = RCSGPR(
        (X_train, Y_train),
        kernel=create_rbf(dataset.train, rng),
        weighting_function=IRQ(C=np.quantile(np.abs(Y_train), 0.9)),
        noise_variance=noise_variance ,
        inducing_variable = inducing_variable
    )
        gpflow.set_trainable(model.likelihood.variance, False)

        opt = gpflow.optimizers.Scipy()
        log = opt.minimize(model.training_loss_closure(), model.trainable_variables,options={"maxiter": 1_000})
        t_after = perf_counter()

        time_no_outliers[dataset.name]['RCSVGP'].append(t_after-t_before)

        mean, _  = model.predict_f(X_test)
        mae_no_outliers[dataset.name]['RCSVGP'].append(np.average(np.abs(mean-Y_test)))



boston
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
energy
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
synthetic
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
yacht
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [33]:
models = ['SVGP', 'RCSVGP']
for dataset in no_outliers:
    for model in models:
        print('model:{}, dataset: {}, time: {} ({})'.format(model, dataset.name, np.average(time_no_outliers[dataset.name][model]),np.std(time_no_outliers[dataset.name][model])))
print('----------------------------------------------------------------------------------------------------------------------')
for dataset in no_outliers:
    for model in models:
        print('model:{}, dataset: {}, mae: {} ({})'.format(model, dataset.name, np.average(mae_no_outliers[dataset.name][model]),np.std(mae_no_outliers[dataset.name][model])))

model:SVGP, dataset: boston, time: 1.1524428709060885 (0.04044354858509785)
model:RCSVGP, dataset: boston, time: 3.8201977706921753 (5.357032944975014)
model:SVGP, dataset: energy, time: 17.22296053741011 (1.1048802735625651)
model:RCSVGP, dataset: energy, time: 10.91163175419788 (4.977299517648173)
model:SVGP, dataset: synthetic, time: 0.7055033833079506 (0.07201487117017041)
model:RCSVGP, dataset: synthetic, time: 0.9406442665960639 (0.10974695481237333)
model:SVGP, dataset: yacht, time: 10.731899291701847 (0.7533074455622992)
model:RCSVGP, dataset: yacht, time: 4.66001008329913 (3.3528505350417044)
----------------------------------------------------------------------------------------------------------------------
model:SVGP, dataset: boston, mae: 0.6568492462534787 (2.6235260252243833e-13)
model:RCSVGP, dataset: boston, mae: 0.5642291486194168 (0.18527725436183426)
model:SVGP, dataset: energy, mae: 0.02967809224549133 (0.00015243907999111473)
model:RCSVGP, dataset: energy, mae: 0.

In [34]:
focused = [ds.boston_focused.create_dataset(Path('/tmp/benchmark_data')), 
               ds.energy_focused.create_dataset(Path('/tmp/benchmark_data')),
                ds.synthetic_focused.create_dataset(Path('/tmp/benchmark_data')),
                  ds.yacht_focused.create_dataset(Path('/tmp/benchmark_data'))]
time_focused ={}
mae_focused = {}

In [35]:
for dataset in focused:
    print(dataset.name)
    time_focused[dataset.name] = {'SVGP':[],'RCSVGP':[]}
    mae_focused[dataset.name] = {'SVGP':[],'RCSVGP':[]}
    for i in range(10):
        print('Iteration',i)
        X_train = dataset.train.X
        Y_train = dataset.train.Y
        X_test = dataset.test.X
        Y_test = dataset.test.Y

        rng = np.random.default_rng(1235+i)

        inducing_variable = create_inducing(dataset.train)
        variance=rng.gamma(5.0, 0.2, []),
        lengthscales=rng.gamma(5.0, 0.2, [dataset.train.D]),
        noise_variance = 0.01

        t_before = perf_counter()
        model = gpflow.models.SGPR(
        (X_train, Y_train),
        kernel=create_rbf(dataset.train, rng),
        noise_variance=noise_variance,
        inducing_variable = inducing_variable
    )
        opt = gpflow.optimizers.Scipy()
        log = opt.minimize(model.training_loss_closure(), model.trainable_variables,options={"maxiter": 1_000})
        t_after = perf_counter()
        time_focused[dataset.name]['SVGP'].append(t_after-t_before)

        mean, _  = model.predict_f(X_test)
        mae_focused[dataset.name]['SVGP'].append(np.average(np.abs(mean-Y_test)))

        t_before = perf_counter()
        model = RCSGPR(
        (X_train, Y_train),
        kernel=create_rbf(dataset.train, rng),
        weighting_function=IMQ(C=np.quantile(np.abs(Y_train), 0.9)),
        noise_variance=noise_variance ,
        inducing_variable = inducing_variable
    )
        gpflow.set_trainable(model.likelihood.variance, False)

        opt = gpflow.optimizers.Scipy()
        log = opt.minimize(model.training_loss_closure(), model.trainable_variables,options={"maxiter": 1_000})
        t_after = perf_counter()

        time_focused[dataset.name]['RCSVGP'].append(t_after-t_before)

        mean, _  = model.predict_f(X_test)
        mae_focused[dataset.name]['RCSVGP'].append(np.average(np.abs(mean-Y_test)))


boston_focused
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
energy_focused
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
synthetic_focused
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
yacht_focused
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [36]:
models = ['SVGP', 'RCSVGP']
for dataset in focused:
    for model in models:
        print('model:{}, dataset: {}, time: {} ({})'.format(model, dataset.name, np.average(time_focused[dataset.name][model]),np.std(time_focused[dataset.name][model])))
print('----------------------------------------------------------------------------------------------------------------------')
for dataset in focused:
    for model in models:
        print('model:{}, dataset: {}, mae: {} ({})'.format(model, dataset.name, np.average(mae_focused[dataset.name][model]),np.std(mae_focused[dataset.name][model])))

model:SVGP, dataset: boston_focused, time: 2.1571872083004564 (3.959837502111929)
model:RCSVGP, dataset: boston_focused, time: 2.9828643665998245 (4.294975308547808)
model:SVGP, dataset: energy_focused, time: 14.723021258303197 (0.7223104894627002)
model:RCSVGP, dataset: energy_focused, time: 12.264381703990512 (0.7951027095703415)
model:SVGP, dataset: synthetic_focused, time: 0.6373753373976797 (0.10030707687854683)
model:RCSVGP, dataset: synthetic_focused, time: 0.8432655624928884 (0.1240460869352342)
model:SVGP, dataset: yacht_focused, time: 9.84658174579381 (0.34200220408883375)
model:RCSVGP, dataset: yacht_focused, time: 10.114171916700434 (0.49627061420662216)
----------------------------------------------------------------------------------------------------------------------
model:SVGP, dataset: boston_focused, mae: 0.6472104312685494 (0.02891644495458125)
model:RCSVGP, dataset: boston_focused, mae: 0.5506469883283243 (0.1679913906633291)
model:SVGP, dataset: energy_focused, ma

In [37]:
asymmetric = [ds.boston_asymmetric.create_dataset(Path('/tmp/benchmark_data')), 
               ds.energy_asymmetric.create_dataset(Path('/tmp/benchmark_data')),
                ds.synthetic_asymmetric.create_dataset(Path('/tmp/benchmark_data')),
                  ds.yacht_asymmetric.create_dataset(Path('/tmp/benchmark_data'))]
time_asymmetric ={}
mae_asymmetric = {}

In [40]:
for dataset in asymmetric:
    print(dataset.name)
    time_asymmetric[dataset.name] = {'SVGP':[],'RCSVGP':[]}
    mae_asymmetric[dataset.name] = {'SVGP':[],'RCSVGP':[]}
    for i in range(10):
        print('Iteration',i)
        X_train = dataset.train.X
        Y_train = dataset.train.Y
        X_test = dataset.test.X
        Y_test = dataset.test.Y

        rng = np.random.default_rng(1235+i)

        inducing_variable = create_inducing(dataset.train)
        variance=rng.gamma(5.0, 0.2, []),
        lengthscales=rng.gamma(5.0, 0.2, [dataset.train.D]),
        noise_variance = 0.01

        t_before = perf_counter()
        model = gpflow.models.SGPR(
        (X_train, Y_train),
        kernel=create_rbf(dataset.train, rng),
        noise_variance=noise_variance,
        inducing_variable = inducing_variable
    )
        opt = gpflow.optimizers.Scipy()
        log = opt.minimize(model.training_loss_closure(), model.trainable_variables,options={"maxiter": 1_000})
        t_after = perf_counter()
        time_asymmetric[dataset.name]['SVGP'].append(t_after-t_before)

        mean, _  = model.predict_f(X_test)
        mae_asymmetric[dataset.name]['SVGP'].append(np.average(np.abs(mean-Y_test)))

        t_before = perf_counter()
        model = RCSGPR(
        (X_train, Y_train),
        kernel=create_rbf(dataset.train, rng),
        weighting_function=IMQ(C=np.quantile(np.abs(Y_train), 0.85)),
        noise_variance=noise_variance ,
        inducing_variable = inducing_variable
    )
        gpflow.set_trainable(model.likelihood.variance, False)

        opt = gpflow.optimizers.Scipy()
        log = opt.minimize(model.training_loss_closure(), model.trainable_variables,options={"maxiter": 1_000})
        t_after = perf_counter()

        time_asymmetric[dataset.name]['RCSVGP'].append(t_after-t_before)

        mean, _  = model.predict_f(X_test)
        mae_asymmetric[dataset.name]['RCSVGP'].append(np.average(np.abs(mean-Y_test)))


boston_asymmetric
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
energy_asymmetric
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
synthetic_asymmetric
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
yacht_asymmetric
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [41]:
models = ['SVGP', 'RCSVGP']
for dataset in asymmetric:
    for model in models:
        print('model:{}, dataset: {}, time: {} ({})'.format(model, dataset.name, np.average(time_asymmetric[dataset.name][model]),np.std(time_asymmetric[dataset.name][model])))
print('----------------------------------------------------------------------------------------------------------------------')
for dataset in asymmetric:
    for model in models:
        print('model:{}, dataset: {}, mae: {} ({})'.format(model, dataset.name, np.average(mae_asymmetric[dataset.name][model]),np.std(mae_asymmetric[dataset.name][model])))

model:SVGP, dataset: boston_asymmetric, time: 0.8937575625081081 (0.09855557096134779)
model:RCSVGP, dataset: boston_asymmetric, time: 2.9952321917982774 (4.452993468908787)
model:SVGP, dataset: energy_asymmetric, time: 1.622505891614128 (0.235123065948699)
model:RCSVGP, dataset: energy_asymmetric, time: 12.835730562504613 (0.44733740793814675)
model:SVGP, dataset: synthetic_asymmetric, time: 0.6461752126924694 (0.028499816047371233)
model:RCSVGP, dataset: synthetic_asymmetric, time: 2.913500241591828 (0.7120716923927498)
model:SVGP, dataset: yacht_asymmetric, time: 2.064033645991003 (1.8525260935095944)
model:RCSVGP, dataset: yacht_asymmetric, time: 10.145659875107231 (0.4904374962637686)
----------------------------------------------------------------------------------------------------------------------
model:SVGP, dataset: boston_asymmetric, mae: 0.656849246253272 (6.620513056479423e-13)
model:RCSVGP, dataset: boston_asymmetric, mae: 0.6494647175419729 (0.14346494796850884)
model:S